In [ ]:
from kami_pricing_analytics.data_collector.strategies.web_scraping.utils import group_user_agents_by_content_similarity, group_user_agents_by_identical_content
from kami_pricing_analytics.data_collector.strategies.web_scraping.constants import USER_AGENTS
url = "https://produto.mercadolivre.com.br/MLB-1448733946"

In [ ]:
similar_groups = group_user_agents_by_content_similarity(USER_AGENTS, url)
identical_groups = group_user_agents_by_identical_content(USER_AGENTS, url)

In [ ]:
for group in similar_groups:
    print('Group:')
    for agent in group:
        print(agent)
    print('\n')

In [ ]:
for group in identical_groups:
    print('Group:')
    for agent in group:
        print(agent)
    print('\n')